# 08 - Uncertainty Simulation

Proposito: ejecutar transformacion normal score y simulacion secuencial gaussiana (SGS) en una malla de bloques.

Outputs esperados:
- `outputs/tables/sgs_realizations.csv`
- `outputs/tables/sgs_quantiles.csv`


### 1. Project setup


In [ ]:
import os, sys, json
import pandas as pd
from IPython.display import display

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()
    os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print('PROJECT_ROOT:', PROJECT_ROOT)


### 2. Configuracion (opcional)


In [ ]:
config_path = 'config/project.json'
with open(config_path, 'r', encoding='utf-8') as f:
    cfg = json.load(f)

simulation_cfg = {
    'enabled': True,
    'n_realizations': 20,
    'seed': 2024,
    'search_radius': cfg.get('kriging', {}).get('search_radius', 150.0),
    'min_samples': cfg.get('kriging', {}).get('min_samples', 4),
    'max_samples': cfg.get('kriging', {}).get('max_samples', 12),
}
simulation_cfg


### 3. Preparar datos, grilla y variograma


In [ ]:
from src.preprocess import load_and_preprocess
from src.grid import grid_from_extents, make_grid_dataframe
from src.variography import experimental_variogram_2d, fit_variogram_model

df, _df_raw, _mapping = load_and_preprocess(cfg)

grid_cfg = cfg.get('grid', {})
if grid_cfg.get('auto_from_data', True):
    grid_spec = grid_from_extents(
        df,
        'x',
        'y',
        'z',
        float(grid_cfg.get('dx', 25.0)),
        float(grid_cfg.get('dy', 25.0)),
        float(grid_cfg.get('dz', 5.0)),
        pad=float(grid_cfg.get('pad', 0.0)),
    )
else:
    grid_spec = {
        'nx': grid_cfg['nx'],
        'ny': grid_cfg['ny'],
        'nz': grid_cfg.get('nz', 1),
        'xmin': grid_cfg['xmin'],
        'ymin': grid_cfg['ymin'],
        'zmin': grid_cfg.get('zmin', 0.0),
        'dx': grid_cfg['dx'],
        'dy': grid_cfg['dy'],
        'dz': grid_cfg.get('dz', 1.0),
    }

grid_df = make_grid_dataframe(grid_spec)

vario_params = cfg.get('variogram', {})
exp = experimental_variogram_2d(df, 'x', 'y', 'var', vario_params)
vario_model = fit_variogram_model(exp, float(df['var'].var(ddof=1)), model_type='spherical')

grid_df.head()


### 4. Ejecutar SGS y resumir P10/P50/P90


In [ ]:
from src.geostats_pipeline.simulation import sgs_block_simulation

if simulation_cfg.get('enabled', True):
    realizations_df, quantiles_df = sgs_block_simulation(
        df,
        grid_df,
        vario_model,
        params=simulation_cfg,
        n_realizations=int(simulation_cfg.get('n_realizations', 20)),
        seed=simulation_cfg.get('seed', 2024),
    )

    os.makedirs('outputs/tables', exist_ok=True)
    realizations_df.to_csv('outputs/tables/sgs_realizations.csv', index=False)
    quantiles_df.to_csv('outputs/tables/sgs_quantiles.csv', index=False)

    display(quantiles_df.head())
else:
    print('SGS deshabilitado: simulation_cfg[enabled] = False')
